In [5]:
from scipy import constants as sci

In [7]:
import mendeleev as pt
def awgh(x):
    return x.atomic_weight * si.gram / si.mole

In [8]:
# constants
class phys:
    g     = sci.g * si.metre / si.second**2
    pi    = sci.pi
    R_str = sci.R * si.joule / si.kelvin / si.mole       


    c_pd  = 1005 * si.joule / si.kilogram / si.kelvin
    c_pv  = 1850 * si.joule / si.kilogram / si.kelvin
    c_pw  = 4218 * si.joule / si.kilogram / si.kelvin
    
    p_tri = 611.73  * si.pascal
    T_tri = 273.16  * si.kelvin
    l_tri = 2.5e6   * si.joule / si.kilogram
    
    sgm   = 0.072   * si.joule / si.metre**2
    rho_w = 1 * si.kilograms / si.litres
    
    Md    = 0.78 * awgh(pt.N) * 2 + 0.21 * awgh(pt.O) * 2 + 0.01 * awgh(pt.Ar)
    Mv    = awgh(pt.O) + awgh(pt.H) * 2
    eps   = Mv / Md
    Rd    = R_str / Md
    Rv    = R_str / Mv
    D0    = 2.26e-5 * si.metre**2 / si.second
    K0    = 2.4e-2  * si.joules / si.metres / si.seconds / si.kelvins
    
    
# pressure deriv. (hydrostatic) 
phys.dp_dt = lambda p, T, w, q: -(p / phys.R(q) / T) * phys.g * w


# temperature deriv. (adiabatic)
phys.dT_dt = lambda T,  p, dp_dt, q, dq_dt : (T * phys.R(q) / p * dp_dt - phys.lv(T)*dq_dt ) / phys.c_p(q)


# specific humidity deriv.
phys.dq_dt = lambda n, r, dr_dt: -4 * sci.pi * np.sum(n * r**2 * dr_dt) * phys.rho_w

# funkcja lambda wystarczy zdefiniować jedną i odwoływać się do niej
phys.dr_dt = lambda r, T, RH, kp, rd: 1/r * (RH - 1 - phys.A(T)/r + phys.B(kp,rd)/r**3) / (phys.Fd(T)+phys.Fk(T,phys.lv(T)))

# RH from specific humidity, temperature and pressure
phys.RH = lambda T, p, q: p * q / (phys.eps + q * (1 - phys.eps)) / phys.pvs(T)

#
phys.mix = lambda q, dry, wet: wet/(1/q + 1) + dry/(1 + q)
phys.c_p = lambda q: phys.mix(q, phys.c_pd, phys.c_pv)
phys.R   = lambda q: phys.mix(q, phys.Rd, phys.Rv)

# latent heat of evaporation 
phys.lv  = lambda T: phys.l_tri + (phys.c_pv - phys.c_pw) * (T - phys.T_tri)

# Maxwel-Mason coefficients
phys.Fd = lambda T: phys.rho_w * phys.Rv * T / phys.D0 / phys.pvs(T)
phys.Fk = lambda T, lv: phys.rho_w * lv / phys.K0 / T * (lv / phys.Rv / T - 1)

# Koehler curve (expressed in partial pressure)      <--- co z rd?
phys.A = lambda T: 2 * phys.sgm / phys.Rv / T / phys.rho_w
phys.B = lambda kp, rd: kp * rd**3

# critical radius
phys.r_cr = lambda kp, rd, T: sqrt(3 * kp * rd**3 / phys.A(T))


In [9]:
# integrated Clausius-Clapeyron eq.
phys.pvs = lambda T: phys.p_tri * exp(
    (phys.l_tri + (phys.c_pw - phys.c_pv) * phys.T_tri) / phys.Rv * (1 / phys.T_tri - 1/T) - 
    (phys.c_pw - phys.c_pv) / phys.Rv * log(T / phys.T_tri)
)

# latent heat of evaporation 
phys.lv  = lambda T: phys.l_tri + (phys.c_pv - phys.c_pw) * (T - phys.T_tri)